# MongoDB arXiv Project

Este notebook contiene el código y la documentación para interactuar con la base de datos MongoDB utilizando PyMongo. A continuación, se presentan las secciones para cada uno de los requerimientos de la tarea.

In [7]:
# Importar las bibliotecas necesarias
from pymongo import MongoClient
import json

# Conectar al Replica Set de MongoDB usando una configuración funcional
try:
    client = MongoClient('mongodb://127.0.0.1:30001,127.0.0.1:30002,127.0.0.1:30003/?replicaSet=my-replica-set&readPreference=primary')
    db = client['arxiv']
    articles_collection = db['articles']
    print('Conexión exitosa al Replica Set de MongoDB')
except Exception as e:
    print(f"Error al conectar: {e}")

Conexión exitosa al Replica Set de MongoDB


## Cargar datos desde el archivo JSON

A continuación, cargaremos los datos desde el archivo `uarxiv-rol1-rol2.json` y los insertaremos en la colección `articles`.

In [8]:
# Cargar datos desde el archivo JSON
try:
    with open('../data/uarxiv-rol1-rol2.json') as f:
        data = json.load(f)
    if not isinstance(data, list) or not data:
        raise ValueError("El archivo JSON debe contener una lista no vacía.")
    # Insertar datos en la colección
    articles_collection.insert_many(data)
    print(f'Documentos insertados: {articles_collection.count_documents({})}')
except FileNotFoundError:
    print("El archivo JSON no se encuentra en la ruta especificada.")
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error inesperado: {e}")

Error: El archivo JSON debe contener una lista no vacía.


## Consultas a la base de datos

Realizaremos algunas consultas para verificar que los datos se han insertado correctamente.

In [9]:
# Consultar los primeros 5 documentos
first_five_articles = articles_collection.find().limit(5)
for article in first_five_articles:
    print(article)

ServerSelectionTimeoutError: mongo2:30002: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:30003: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo1:30001: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 683647d0dbf92a76ef87b812, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:30001: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('mongo2', 30002) server_type: Unknown, rtt: None, error=AutoReconnect('mongo2:30002: [Errno 11001] getaddrinfo failed (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescriptio

## Actualizar documentos

A continuación, realizaremos una actualización en uno de los documentos.

In [ ]:
# Actualizar un documento
articles_collection.update_one({'id': 'some_id'}, {'$set': {'title': 'Nuevo Título'}})

# Verificar la actualización
updated_article = articles_collection.find_one({'id': 'some_id'})
print(updated_article)

## Eliminar documentos

Finalmente, eliminaremos un documento de la colección.

In [ ]:
# Eliminar un documento
articles_collection.delete_one({'id': 'some_id'})

# Verificar la eliminación
print(f'Documentos restantes: {articles_collection.count_documents({})}')

## Conclusiones

En este notebook, hemos realizado operaciones básicas de CRUD en la base de datos MongoDB utilizando PyMongo. Se han cargado datos, realizado consultas, actualizaciones y eliminaciones.

## Consultar artículos publicados en 2025

Devolver los títulos y fechas de creación de artículos publicados en el año 2025. Mostrar solo esos campos y limitar a los primeros 20 resultados.

In [ ]:
query_a = {"published": {"$regex": "^2025"}}
results_a = articles_collection.find(query_a, {"title": 1, "published": 1}).limit(20)
for article in results_a:
    print(article)

## Consultar artículos en categorías 'cs.AI' o 'stat.ML' con al menos tres autores

Devolver los títulos y los autores de artículos que pertenezcan a las categorías "cs.AI" o "stat.ML" y que tengan al menos tres autores. Mostrar solo esos campos y limitar a los primeros 10 resultados.

In [ ]:
query_b = {"categories": {"$in": ["cs.AI", "stat.ML"]}, "authors": {"$size": {"$gte": 3}}}
results_b = articles_collection.find(query_b, {"title": 1, "authors": 1}).limit(10)
for article in results_b:
    print(article)

## Demostrar consistencia en los nodos replicados

Insertar un nuevo documento en el maestro y verificar que se replica correctamente en los nodos secundarios.

In [ ]:
# Insertar un nuevo documento
new_document = {"id": "test_id", "title": "Documento de prueba", "published": "2025-05-27"}
articles_collection.insert_one(new_document)

# Verificar replicación en nodos secundarios
replicated_document = articles_collection.find_one({"id": "test_id"})
print(replicated_document)

## Demostrar alta disponibilidad

Apagar el nodo maestro y verificar que los nodos secundarios responden correctamente a las consultas.

In [ ]:
# Simular alta disponibilidad
# Realizar una consulta mientras el nodo maestro está apagado
available_documents = articles_collection.find().limit(5)
for document in available_documents:
    print(document)

## Consultar artículos en la categoría 'hep-ph' con DOI asignado

Devolver los títulos, las categorías y los enlaces al PDF de artículos que pertenezcan a la categoría "hep-ph" y tengan un DOI asignado. Mostrar solo esos campos y limitar a 15 resultados.

In [ ]:
query_c = {"categories": "hep-ph", "doi": {"$exists": True}}
results_c = articles_collection.find(query_c, {"title": 1, "categories": 1, "pdf_link": 1}).limit(15)
for article in results_c:
    print(article)

## Consultar artículos con DOI asignado, ordenados por título

Devolver los títulos, nombres de los autores y la referencia de publicación (journal-ref) de los artículos que tengan un DOI asignado. Mostrar solo esos campos y ordenar los resultados alfabéticamente por título. Limitar a los primeros 20 resultados.

In [ ]:
query_d = {"doi": {"$exists": True}}
results_d = articles_collection.find(query_d, {"title": 1, "authors": 1, "journal-ref": 1}).sort("title", 1).limit(20)
for article in results_d:
    print(article)

## Consultar artículos enviados entre los años 2010 y 2015

Devolver los títulos y la fecha de la primera versión (versions.created) de los artículos enviados entre los años 2010 y 2015. Mostrar solo esos campos y limitar a los primeros 15 resultados.

In [ ]:
query_e = {"versions.created": {"$gte": "2010", "$lte": "2015"}}
results_e = articles_collection.find(query_e, {"title": 1, "versions.created": 1}).limit(15)
for article in results_e:
    print(article)

## Consultar artículos con comentarios definidos y no nulos

Devolver los títulos, comentarios y reportes técnicos (report-no) de artículos que tengan comentarios definidos y no nulos. Mostrar solo esos campos, ordenando por fecha de actualización (update_date) en orden descendente. Limitar a 10 resultados.

In [ ]:
query_f = {"comments": {"$exists": True, "$ne": None}}
results_f = articles_collection.find(query_f, {"title": 1, "comments": 1, "report-no": 1}).sort("update_date", -1).limit(10)
for article in results_f:
    print(article)

## Consultar artículos con DOI asignado, ordenados por título

Devolver los títulos, nombres de los autores y la referencia de publicación (journal-ref) de los artículos que tengan un DOI asignado. Mostrar solo esos campos y ordenar los resultados alfabéticamente por título. Limitar a los primeros 20 resultados.

In [ ]:
query_d = {"doi": {"$exists": True}}
results_d = articles_collection.find(query_d, {"title": 1, "authors": 1, "journal-ref": 1}).sort("title", 1).limit(20)
for article in results_d:
    print(article)

## Consultar artículos enviados entre los años 2010 y 2015

Devolver los títulos y la fecha de la primera versión (versions.created) de los artículos enviados entre los años 2010 y 2015. Mostrar solo esos campos y limitar a los primeros 15 resultados.

In [ ]:
query_e = {"versions.created": {"$gte": "2010", "$lte": "2015"}}
results_e = articles_collection.find(query_e, {"title": 1, "versions.created": 1}).limit(15)
for article in results_e:
    print(article)

## Consultar artículos con comentarios definidos y no nulos

Devolver los títulos, comentarios y reportes técnicos (report-no) de artículos que tengan comentarios definidos y no nulos. Mostrar solo esos campos, ordenando por fecha de actualización (update_date) en orden descendente. Limitar a 10 resultados.

In [ ]:
query_f = {"comments": {"$exists": True, "$ne": None}}
results_f = articles_collection.find(query_f, {"title": 1, "comments": 1, "report-no": 1}).sort("update_date", -1).limit(10)
for article in results_f:
    print(article)

## Consultar artículos enviados entre los años 2010 y 2015

Devolver los títulos y la fecha de la primera versión (versions.created) de los artículos enviados entre los años 2010 y 2015. Mostrar solo esos campos y limitar a los primeros 15 resultados.

In [ ]:
query_e = {"versions.created": {"$gte": "2010", "$lte": "2015"}}
results_e = articles_collection.find(query_e, {"title": 1, "versions.created": 1}).limit(15)
for article in results_e:
    print(article)

## Consultar artículos con comentarios definidos y no nulos

Devolver los títulos, comentarios y reportes técnicos (report-no) de artículos que tengan comentarios definidos y no nulos. Mostrar solo esos campos, ordenando por fecha de actualización (update_date) en orden descendente. Limitar a 10 resultados.

In [ ]:
query_f = {"comments": {"$exists": True, "$ne": None}}
results_f = articles_collection.find(query_f, {"title": 1, "comments": 1, "report-no": 1}).sort("update_date", -1).limit(10)
for article in results_f:
    print(article)

## Consultar artículos enviados entre los años 2010 y 2015

Devolver los títulos y la fecha de la primera versión (versions.created) de los artículos enviados entre los años 2010 y 2015. Mostrar solo esos campos y limitar a los primeros 15 resultados.

In [ ]:
query_e = {"versions.created": {"$gte": "2010", "$lte": "2015"}}
results_e = articles_collection.find(query_e, {"title": 1, "versions.created": 1}).limit(15)
for article in results_e:
    print(article)

## Consultar artículos con comentarios definidos y no nulos

Devolver los títulos, comentarios y reportes técnicos (report-no) de artículos que tengan comentarios definidos y no nulos. Mostrar solo esos campos, ordenando por fecha de actualización (update_date) en orden descendente. Limitar a 10 resultados.

In [ ]:
query_f = {"comments": {"$exists": True, "$ne": None}}
results_f = articles_collection.find(query_f, {"title": 1, "comments": 1, "report-no": 1}).sort("update_date", -1).limit(10)
for article in results_f:
    print(article)

## Demostrar consistencia en los nodos replicados

Insertar un nuevo documento en el maestro y verificar que se replica correctamente en los nodos secundarios.

In [ ]:
# Insertar un nuevo documento
new_document = {"id": "test_id", "title": "Documento de prueba", "published": "2025-05-27"}
articles_collection.insert_one(new_document)

# Verificar replicación en nodos secundarios
replicated_document = articles_collection.find_one({"id": "test_id"})
print(replicated_document)

## Demostrar alta disponibilidad

Apagar el nodo maestro y verificar que los nodos secundarios responden correctamente a las consultas.

In [ ]:
# Ejecutar servicios de Docker Compose
import os
os.chdir(r'c:\Users\maxim\Desktop\bd\mongodb-arxiv-project\src\docker')
os.system('docker-compose up -d')

In [ ]:
# Simular alta disponibilidad
# Realizar una consulta mientras el nodo maestro está apagado
available_documents = articles_collection.find().limit(5)
for document in available_documents:
    print(document)

In [ ]:
# Verificar la versión de ipykernel
import ipykernel
print(ipykernel.__version__)